<a href="https://colab.research.google.com/github/ssttv/ml-resources/blob/master/fastai_rf_interpretation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!pip3 install fastai==0.7.0

    100% |████████████████████████████████| 122kB 3.6MB/s 
    100% |████████████████████████████████| 2.2MB 12.6MB/s 
    100% |████████████████████████████████| 496.4MB 31kB/s 
    100% |████████████████████████████████| 71kB 3.9MB/s 
    100% |████████████████████████████████| 112kB 26.9MB/s 
    100% |████████████████████████████████| 3.6MB 2.6MB/s 
    100% |████████████████████████████████| 184kB 5.4MB/s 
    100% |████████████████████████████████| 1.5MB 4.5MB/s 
    100% |████████████████████████████████| 122kB 27.2MB/s 
    100% |████████████████████████████████| 11.6MB 2.3MB/s 
    100% |████████████████████████████████| 61kB 20.6MB/s 
    100% |████████████████████████████████| 92kB 27.8MB/s 
    100% |████████████████████████████████| 122kB 30.4MB/s 
    100% |████████████████████████████████| 81kB 21.2MB/s 
  Running setup.py bdist_wheel for feather-format ... - \ done
  Stored in directory: /root/.cache/pip/wheels/85/7d/12/2dfa5c0195f921ac935f5e8f27deada74972edc0ae9988

In [0]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
PATH = '/content/drive/generic_datasets/bulldozer/'

In [0]:
df_raw = pd.read_csv(f'{PATH}Train.csv', low_memory=False, 
                     parse_dates=["saledate"])